# Atividade MC 2 - Implementação de Gradiente Descendente e Redes Neurais

Este notebook contém a resolução dos exercícios propostos na "MC Atividade 2". O foco é traduzir a lógica visual dos slides (StatQuest) para código Python funcional.

Bibliotecas necessárias:

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import math

# Configuração para melhorar a visualização dos gráficos
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## Questão A: Gradiente Descendente (Regressão Linear)

**Objetivo:** Implementar o procedimento dos slides 156-213.
* **Dados:** Altura vs Peso.
* **Modelo:** `Altura = Intercepto + 0.64 * Peso` (O declive/slope é fixo em 0.64).
* **Tarefa:** Otimizar o **Intercepto** começando de 0.
* **Visualização:** Mostrar o movimento da reta e a descida na curva de custo (SSR).
* **Condições de parada:** Número máximo de iterações OU Step Size muito pequeno.

In [ ]:
def run_question_a(learning_rate, max_iter=20, tolerance=0.001):
    #dados do slide 
    weights = np.array([0.5, 2.3, 2.9])
    heights = np.array([1.4, 1.9, 3.2])
    
    #parâmetros Iniciais que foram definidos no slide
    slope_fixed = 0.64
    intercept = 0.0  # Começa em 0 conforme o slide
    
    #para plotagem da curva de custo
    intercept_vals = np.linspace(-0.5, 1.5, 100)
    ssr_vals = []
    for i in intercept_vals:
        predicted = i + slope_fixed * weights
        residuals = heights - predicted
        ssr_vals.append(np.sum(residuals**2))
        
    print(f"\n--- Iniciando Gradiente Descendente (LR: {learning_rate}) ---")
    
    #loop para otimização do intercepto
    for i in range(max_iter):
        #calcular Predições e Derivada
        predicted_heights = intercept + slope_fixed * weights
        
        #calculo do predicted e da derivada
        residuals = heights - predicted_heights
        derivative = np.sum(-2 * residuals)
        
        # calcular Step Size
        step_size = derivative * learning_rate
        
        #guardar valor antigo para impressão
        old_intercept = intercept
        
        #atualizar intercept
        new_intercept = old_intercept - step_size
        
        print(f"Iter {i+1}: Old Intercept={old_intercept:.3f}, Step Size={step_size:.3f}, New Intercept={new_intercept:.3f}, Slope(Grad)={derivative:.3f}")
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        #grafico da reta de regressão
        axes[0].scatter(weights, heights, color='green', s=100, label='Dados Reais')
        axes[0].plot(weights, predicted_heights, color='teal', linewidth=2, label=f'Reta (Intercept={old_intercept:.2f})')

        for w, h, p in zip(weights, heights, predicted_heights):
            axes[0].plot([w, w], [h, p], color='red', linestyle='--')
        axes[0].set_title(f"Iteração {i+1}: Ajuste da Reta")
        axes[0].set_xlabel("Peso")
        axes[0].set_ylabel("Altura")
        axes[0].legend()
        axes[0].set_ylim(0, 4)
        
        # Gráfico da curva de custo
        current_ssr = np.sum((heights - (old_intercept + slope_fixed * weights))**2)
        axes[1].plot(intercept_vals, ssr_vals, color='teal')
        axes[1].scatter(old_intercept, current_ssr, color='red', s=100, zorder=5)
        #tangente (visualização do gradiente)
        tangent_line = derivative * (intercept_vals - old_intercept) + current_ssr
        axes[1].plot(intercept_vals, tangent_line, color='orange', linestyle='--', alpha=0.5, label='Inclinação (Derivada)')
        
        axes[1].set_title(f"Soma dos Resíduos ao Quadrado vs Intercepto")
        axes[1].set_xlabel("Intercepto")
        axes[1].set_ylabel("SSR")
        axes[1].legend()
        
        plt.tight_layout()
        plt.show()
        
        intercept = new_intercept
        
        #condição de parada 2: step size muito pequeno
        if abs(step_size) < tolerance:
            print("-> Convergência alcançada (Step Size pequeno).")
            break
            
    print(f"Resultado Final para LR {learning_rate}: Intercepto = {intercept:.4f}")


run_question_a(learning_rate=0.1)

run_question_a(learning_rate=0.01)

## Questão B: Gradiente Descendente Estocástico e Mini-Batch

**Objetivo:** Transformar o procedimento dos slides 302-335 em código.
* **Diferença:** Não atualizar usando a soma de *todos* os dados de uma vez, mas sim amostra por amostra (Estocástico) ou em pequenos grupos (Mini-batch).
* **Nota:** O gráfico complexo da Questão A não será gerado aqui, focaremos na convergência.

### B-1: Gradiente Descendente Estocástico (SGD)
Atualiza o intercepto após calcular o erro para **cada** amostra individualmente.

In [6]:
def run_sgd(learning_rate=0.1, epochs=5):
    weights = np.array([0.5, 2.3, 2.9])
    heights = np.array([1.4, 1.9, 3.2])
    slope_fixed = 0.64
    intercept = 0.0
    
    print(f"--- SGD (Learning Rate: {learning_rate}) ---")
    
    for epoch in range(epochs):
        indices = np.arange(len(weights))
        np.random.shuffle(indices)
        
        print(f"Época {epoch+1}:")
        
        for i in indices:
            w = weights[i]
            h = heights[i]
            
            pred = intercept + slope_fixed * w

            derivative = -2 * (h - pred)

            step_size = derivative * learning_rate
            
            old_intercept = intercept
            intercept = intercept - step_size
            
            print(f"   Amostra (w={w}): Old={old_intercept:.3f}, Step={step_size:.3f}, New={intercept:.3f}")

        return intercept

final_intercept_sgd = run_sgd()
print(f"Final SGD Intercept= {final_intercept_sgd:.4f}")

--- SGD (Learning Rate: 0.1) ---
Época 1:
   Amostra (w=2.3): Old=0.000, Step=-0.086, New=0.086
   Amostra (w=0.5): Old=0.086, Step=-0.199, New=0.284
   Amostra (w=2.9): Old=0.284, Step=-0.212, New=0.496
Final SGD Intercept= 0.4964


### B-2: Gradiente Descendente Mini-Batch
Adaptação para usar **mini-batch de 2 samples**.

In [ ]:
def run_minibatch(learning_rate=0.1, epochs=5, batch_size=2):
    weights = np.array([0.5, 2.3, 2.9])
    heights = np.array([1.4, 1.9, 3.2])
    slope_fixed = 0.64
    intercept = 0.0

    print(f"--- Mini-batch GD (Batch Size: {batch_size}) ---")

    for epoch in range(epochs):
        infices = np.arange(len(weights))
        np.random.shuffle(indices)

        #vai criar as batches
        for start_idx in range(0, len(weights), batch_size):
            end_idx = start_idx + batch_size
            batch_indices = indices[start_idx:end_idx]

            #dados da batch
            w_batch = weights[batch_indices]
            h_batch = heights[batch_indices]

            #predicao e derivada somada para o batch
            pred_batch = intercept + slope_fixed * w_batch
            residuals = h_batch - pred_batch
            derivative = np.sum(-2 * residuals)

            step_size = derivative * learning_rate

            old_intercept = intercept
            intercept = intercept - step_size
